# Readout: Time-Series Analysis

In [ ]:
from cirq_internal import io as ciio
from cirq_internal.benchmarking.experiments.readout_scan.readout \
    import EXPERIMENT_NAME, DEFAULT_BASE_DIR
EXPERIMENT_NAME, DEFAULT_BASE_DIR

In [ ]:
import numpy as np
import pandas as pd

records = []
# Load all data, do some light processing
for record in ciio.iterload_records(dataset_id='*', base_dir=DEFAULT_BASE_DIR):
    
    # Unwrap BitstringArray into np.ndarray
    all_bitstrings = [ba.bitstrings for ba in record['all_bitstrings']]
    
    # Compute <Z>
    record['z_vals'] = [np.mean((-1)**bitstrings, axis=0).item() for bitstrings in all_bitstrings]
    
    # Don't need to carry around the full array of bits anymore
    del record['all_bitstrings']
    records.append(record)
    
df = pd.DataFrame(records)
print(len(df))
df.head()

In [ ]:
# Query relevant data
# Select and drop: device_name, simulator
# Set index

dfb = df
dfb = dfb[dfb['device_name'] == 'Sycamore23']
dfb = dfb.drop('device_name', axis=1)
dfb = dfb[~dfb['simulator']]
dfb = dfb.drop('simulator', axis=1)
dfb['timestamp'] = pd.to_datetime(dfb['timestamp'])
dfb = dfb.sort_values(by=['data_collection_id', 'qubit'])
dfb = dfb.groupby('data_collection_id').filter(lambda x: len(x) > 2)
dfb = dfb.set_index(['data_collection_id', 'qubit'])

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

for dc_id in dfb.index.levels[0]:
    dfc = dfb.loc[dc_id]
    plt.axhline(-1, color='grey')
    plt.axhline(1, color='grey')
    for q, row in dfc.iterrows():
        plt.plot(row['thetas'], row['z_vals'], label=f'{q}')
    plt.ylim(-1.1, 1.1)
        
    #plt.legend(loc='best')
    plt.title(dc_id)
    plt.show()

In [ ]:
eps = 1e-5

p00s = []
p11s = []
for i, row in dfb.iterrows():
    ti1 = np.where(np.abs(np.asarray(row['thetas']) - 0) < eps)[0][0]
    ti2 = np.where(np.abs(np.asarray(row['thetas']) - np.pi)<eps)[0][0]
    p00 = row['z_vals'][ti1]
    p11 = -1*row['z_vals'][ti2]
    p00s.append(p00)
    p11s.append(p11)

dfb['p00'] = p00s
dfb['p11'] = p11s
dfb.head()

In [ ]:
import seaborn as sns
import pandas as pd
dfc = dfb.reset_index()
dfc = pd.melt(dfc, id_vars=['data_collection_id', 'qubit'], value_vars=['p00', 'p11'])

# Outlier
dfc = dfc[dfc['value'] > 0.5]

fig, ax = plt.subplots(figsize=(7,5))
sns.violinplot(dfc['data_collection_id'], dfc['value'], hue=dfc['variable'], split=True, ax=ax, cut=0)
ax.legend(loc='best', fontsize=16)
ax.set_xlabel(None)
ax.set_ylabel(None)
fig.autofmt_xdate()
fig.tight_layout()


In [ ]:

for dc_id in dfb.index.levels[0]:
    dfc = dfb.loc[dc_id]    
    fig, ax = plt.subplots(figsize=(7,5))
    ax.scatter(dfc['timestamp'], dfc['p00'])
    ax.scatter(dfc['timestamp'], dfc['p11'])
    ax.set_xlim((dfc['timestamp'].min(), dfc['timestamp'].max()))
    #fig.autofmt_xdate()
    fig.tight_layout()
    plt.show()